In [1]:
%pip install torch
%pip install torch_geometric
%pip install numpy
%pip install tqdm

import torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Get Data

In [2]:
from data_preparation import load_data_tensor

lr_train, lr_test, hr_train = load_data_tensor("dgl-icl")

lr_X_dim1 = torch.load('model_autoencoder/final_embeddings/encode_lr.pt')
lr_X_dim3 = torch.load('model_autoencoder/final_embeddings/encode_lr_3.pt')
hr_X_dim1 = torch.load('model_autoencoder/final_embeddings/encode_hr.pt')
hr_X_dim3 = torch.load('model_autoencoder/final_embeddings/encode_hr_3.pt')
lr_X_dim1_test = torch.load('model_autoencoder/final_embeddings/encode_lr_test.pt')
hr_X_dim3_test = torch.load('model_autoencoder/final_embeddings/encode_lr_test_3.pt')

## Specify Model

In [3]:
import torch
from torch.nn import Sequential, Linear, ReLU
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import NNConv
from torch_geometric.nn import BatchNorm
import numpy as np
from torch_geometric.data import Data
from torch.utils.data import Dataset


N_SOURCE_NODES = 160
N_TARGET_NODES = 268

class Aligner(torch.nn.Module):
    def __init__(self):
        
        super(Aligner, self).__init__()

        nn = Sequential(Linear(1, N_SOURCE_NODES*N_SOURCE_NODES), ReLU())
        self.conv1 = NNConv(N_SOURCE_NODES, N_SOURCE_NODES, nn, aggr='mean', root_weight=True, bias=True)
        self.conv11 = BatchNorm(N_SOURCE_NODES, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)

        nn = Sequential(Linear(1, N_SOURCE_NODES), ReLU())
        self.conv2 = NNConv(N_SOURCE_NODES, 1, nn, aggr='mean', root_weight=True, bias=True)
        self.conv22 = BatchNorm(1, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)

        nn = Sequential(Linear(1, N_SOURCE_NODES), ReLU())
        self.conv3 = NNConv(1, N_SOURCE_NODES, nn, aggr='mean', root_weight=True, bias=True)
        self.conv33 = BatchNorm(N_SOURCE_NODES, eps=1e-03, momentum=0.1, affine=True, track_running_stats=True)


    def forward(self, x, edge_index, edge_attr):
        x01 = self.conv1(x, edge_index, edge_attr)
        x1 = F.sigmoid(self.conv11(x01))
        x1 = F.dropout(x1, training=self.training)

        x2 = F.sigmoid(self.conv22(self.conv2(x1, edge_index, edge_attr)))
        x2 = F.dropout(x2, training=self.training)

        x3 = torch.cat([F.sigmoid(self.conv33(self.conv3(x2, edge_index, edge_attr))), x1], dim=1)
        x4 = x3[:, 0:N_SOURCE_NODES]
        x5 = x3[:, N_SOURCE_NODES:2*N_SOURCE_NODES]

        x6 = (x4 + x5) / 2
        return x6

def transform_to_data(tri):
    N_SOURCE_NODES = tri.shape[-1]
    edge_attr = torch.Tensor(tri).view(N_SOURCE_NODES**2, 1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    counter = 0
    N_ROI = N_SOURCE_NODES

    pos_edge_index = torch.zeros(2, N_ROI * N_ROI)
    for i in range(N_ROI):
        for j in range(N_ROI):
            pos_edge_index[:, counter] = torch.tensor([i, j])
            counter += 1
            
    x = torch.tensor(tri, dtype=torch.float)
    pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)

    return Data(x=x, pos_edge_index=pos_edge_index, edge_attr=edge_attr)


class GraphPairData(Dataset):
    def __init__(self, lr_data, hr_data):
        assert len(lr_data) == len(hr_data), "The lists must have the same length"
        self.lr_data = lr_data
        self.hr_data = hr_data

    def __len__(self):
        return len(self.lr_data)

    def __getitem__(self, idx):
        source = self.lr_data[idx]
        target = self.hr_data[idx]
        return source, target
    
def Alignment_loss(target, predicted):
    # l_loss1 = torch.abs(nn.KLDivLoss()(F.softmax(zt1), F.softmax(z_s1.t())))

    kl_loss = torch.abs(F.kl_div(F.softmax(target), F.softmax(predicted), None, None, 'sum'))
    return kl_loss



In [6]:
DEVICE = torch.device("cuda")

aligner = Aligner()
aligner.to(DEVICE)
Aligner_optimizer = torch.optim.AdamW(aligner.parameters(), lr=0.025, betas=(0.5, 0.999))


In [5]:
x= torch.tensor([1,2])
print(len(x))

2


## Choo Choo Train

In [6]:
from torch_geometric.loader import DataLoader


lr_data = [transform_to_data(x) for x in lr_train]
hr_data = [transform_to_data(x) for x in hr_train]



/tmp/ipykernel_1396824/1680016639.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(edge_attr, dtype=torch.float)
/tmp/ipykernel_1396824/1680016639.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(tri, dtype=torch.float)
/tmp/ipykernel_1396824/1680016639.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_edge_index = torch.tensor(pos_edge_index, dtype=torch.long)


In [19]:
import pickle

with open('dgl-icl/moritz/lr_data.pkl', 'wb') as file:
    pickle.dump(lr_data, file)
with open('dgl-icl/moritz/hr_data.pkl', 'wb') as file:
    pickle.dump(hr_data, file)

In [8]:
import pickle


with open('dgl-icl/moritz/lr_data.pkl', 'rb') as file:
    lr_data = pickle.load(file)
with open('dgl-icl/moritz/hr_data.pkl', 'rb') as file:
    hr_data = pickle.load(file)

In [9]:
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import gc

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

aligner = Aligner()
aligner.to(DEVICE)
Aligner_optimizer = torch.optim.AdamW(aligner.parameters(), lr=0.025, betas=(0.5, 0.999))


N_TARGET_NODES = 268
nbre_epochs = 50

combined_data = GraphPairData(lr_data, hr_data)


# Create a DataLoader
data_loader = DataLoader(combined_data, batch_size=1, shuffle=True)

X = torch.eye(160)


aligner.train()
for epochs in range(nbre_epochs):
        # Train Generator
        Al_losses = []

        i = 0
        for data_source, data_target in tqdm(data_loader):
            torch.cuda.empty_cache()
            gc.collect()

            # print(i)
            # print(data_source.shape)
            # print(data_target.shape)
            # ************    Domain alignment    ************
            _, edge_index, edge_attr = data_source.x, data_source.pos_edge_index, data_source.edge_attr

            #print(i)
            X = X.to(DEVICE)
            edge_index = edge_index.to(DEVICE)
            edge_attr = edge_attr.to(DEVICE)

            A_output = aligner(X, edge_index, edge_attr)
            #print(A_output.shape)

            target =  data_target.edge_attr.view(N_TARGET_NODES, N_TARGET_NODES).detach().cpu().clone().numpy()
            target_mean = np.mean(target)
            target_std = np.std(target)

            target = torch.zeros((N_SOURCE_NODES, N_SOURCE_NODES))
            upper_tricount = int((N_SOURCE_NODES * (N_SOURCE_NODES - 1))/2)
            target_data = torch.normal(target_mean, target_std, size=(1, upper_tricount))
            target[np.triu_indices(N_SOURCE_NODES, 1)] = target_data
            target = target + target.T

            target = target.to(DEVICE)

            kl_loss = Alignment_loss(target, A_output)
            del A_output
            del target
            gc.collect()

            Al_losses.append(kl_loss)
            i += 1

        torch.cuda.empty_cache()
        Aligner_optimizer.zero_grad()
        Al_losses = torch.mean(torch.stack(Al_losses))
        Al_losses.backward(retain_graph=True)
        Aligner_optimizer.step()

        print("[Epoch: %d]| [Al loss: %f]|" % (epochs, Al_losses))


  0%|          | 0/167 [00:00<?, ?it/s]/tmp/ipykernel_1399819/1680016639.py:84: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  kl_loss = torch.abs(F.kl_div(F.softmax(target), F.softmax(predicted), None, None, 'sum'))
  1%|          | 2/167 [00:01<01:26,  1.91it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.44 GiB. GPU 0 has a total capacity of 11.69 GiB of which 1.70 GiB is free. Including non-PyTorch memory, this process has 9.96 GiB memory in use. Of the allocated memory 7.47 GiB is allocated by PyTorch, and 2.33 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 